In [35]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU, if not CPU
print(f"Code run on : {device}")

Code run on : cpu


- eventuellement ajouter itr dans init pour le mettre comme variable self.itr

In [37]:
def generate_A_H_sol(n:int=300,m:int=600, seed:int=12, bs:int=10) : 
    """Generate A and H

    Args:
        n (int, optional): Number of rows. Defaults to 300.
        m (int, optional): Number of columns. Defaults to 600.
        seed (int, optional): Seed for random. Defaults to 12.

    Returns:
        A, H, W: Matrix A (square matrix), matrix H (random matrix), Matrix with diagonal : eigen values of A
    """
    np.random.seed(seed=seed)
    H = np.random.normal(0, 1.0 / math.sqrt(n), (n, m))
    A = np.dot(H, H.T)
    eig = np.linalg.eig(A)[0] # Eigenvalues
    
    W = torch.Tensor(np.diag(eig)).to(device)  # Define the appropriate 'device'
    H = torch.from_numpy(H).float().to(device)  # Define the appropriate 'device'
    
    print("Condition number, min. and max. eigenvalues of A:")
    print(np.max(eig) / np.min(eig), np.max(eig), np.min(eig))
    
    solution = torch.normal(torch.zeros(bs,n),1.0).to(device).detach()
    y = solution@H.detach()
    
    return A,H, W, solution, y

def decompose_matrix(A:np.array) : 
    # Decmposed matrix calculations
    D = np.diag(np.diag(A)) # diagonal matrix
    L = np.tril(A, -1) #  lower triangular matrix
    U = np.triu(A, 1) # upper triangular matrix
    Dinv = np.linalg.inv(D) # inversion diagonal matrix
    invM = np.linalg.inv(D + L) #  inversion matrix M
    # Convert to Torch tensors and move to device
    A = torch.Tensor(A).to(device) 
    D = torch.Tensor(D).to(device) 
    L = torch.Tensor(L).to(device) 
    U = torch.Tensor(U).to(device) 
    Dinv = torch.Tensor(Dinv).to(device) 
    Minv = torch.Tensor(invM).to(device) 
    
    return A, D, L, U, Dinv, Minv

def model_iterations(total_itr:int, n:int, bs:int, model, solution) : 
    norm_list_model = [] # Initialize the iteration list
    s_hats = []
    for i in range(total_itr+1):
        s_hat, _ = model.iterate(i)
        err = (torch.norm(solution.to(device) - s_hat.to(device))**2).item()/(n*bs)
        
        s_hats.append(s_hat)
        norm_list_model.append(err)
    return s_hats, norm_list_model

class base_model() : 
    def __init__(self, n, A, H, bs, y) : 
        
        self.n = n
        self.H = H
        self.bs = bs
        self.y = y
        
        self.A, self.D, self.L, self.U, self.Dinv, self.Minv = decompose_matrix(A)

class GS(base_model):
    """Class implementing the Gauss-Seidel algorithm for solving a linear system.

    Args:
        num_itr (int): The number of Gauss-Seidel iterations to perform.

    Attributes:
        num_itr (int): The number of Gauss-Seidel iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the Gauss-Seidel iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y):
        """Initialize the Gauss-Seidel solver.

        Args:
            num_itr (int): The number of Gauss-Seidel iterations to perform.

        """
        super(GS, self).__init__(n, A, H, bs, y)
        
    def iterate(self, num_itr):
        """Perform the Gauss-Seidel iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        s = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        for i in range(num_itr):
            temp = -torch.matmul(s, self.U) + yMF
            s = torch.matmul(temp, self.Minv)
            traj.append(s)

        return s, traj


class RI(base_model):
    """Class implementing the Richardson iteration algorithm for solving a linear system.

    Args:
        num_itr (int): The number of Richardson iterations to perform.

    Attributes:
        num_itr (int): The number of Richardson iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the Richardson iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y):
        """Initialize the Richardson iteration solver.

        Args:
            num_itr (int): The number of Richardson iterations to perform.

        """
        super(RI, self).__init__(n, A, H, bs, y)

    def iterate(self, num_itr):
        """Perform the Richardson iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        omega = torch.tensor(0.25)
        s = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        for i in range(num_itr):
            s = s + torch.mul(omega, (yMF - torch.matmul(s, self.A)))
            traj.append(s)

        return s, traj

class Jacobi(base_model):
    """Class implementing the Jacobi iteration algorithm for solving a linear system.

    Args:
        num_itr (int): The number of Jacobi iterations to perform.

    Attributes:
        num_itr (int): The number of Jacobi iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the Jacobi iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y, omega:float=0.2):
        """Initialize the Jacobi iteration solver.

        Args:
            num_itr (int): The number of Jacobi iterations to perform.

        """
        super(Jacobi, self).__init__(n, A, H, bs, y)
        self.omega = torch.tensor(omega)
        
    def iterate(self, num_itr):
        """Perform the Jacobi iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        s = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        for i in range(num_itr):
            temp = torch.matmul(self.Dinv, (self.D - self.A))
            s = torch.matmul(s, temp) + torch.matmul(yMF, self.Dinv)
            traj.append(s)

        return s, traj

class SOR(base_model):
    """Class implementing the Successive Over-Relaxation (SOR) algorithm for solving a linear system.

    Args:
        num_itr (int): The number of SOR iterations to perform.

    Attributes:
        num_itr (int): The number of SOR iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the SOR iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y, omega:float=1.8):
        """Initialize the SOR solver.

        Args:
            num_itr (int): The number of SOR iterations to perform.

        """
        super(SOR, self).__init__(n, A, H, bs, y)
        self.omega = torch.tensor(omega)
        
    def iterate(self, num_itr):
        """Perform the SOR iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        n = self.y.size(1)

        inv_omega = torch.div(1, self.omega)
        invM_sor = torch.linalg.inv(self.D - torch.mul(inv_omega, self.L))

        s = torch.zeros(self.bs, n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector
        
        for i in range(num_itr):
            temp = torch.mul((inv_omega - 1), self.D) + torch.mul(inv_omega, self.U)
            s = torch.matmul(s, torch.matmul(invM_sor, temp)) + torch.matmul(yMF, invM_sor)
            traj.append(s)

        return s, traj

class SOR_CHEBY(base_model):
    """Class implementing the SOR-Chebyshev algorithm for solving a linear system.

    Args:
        num_itr (int): The number of SOR-Chebyshev iterations to perform.

    Attributes:
        num_itr (int): The number of SOR-Chebyshev iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the SOR-Chebyshev iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y, omega:float=1.8, omegaa:float=0.8, gamma:float=0.8):
        """Initialize the SOR-Chebyshev solver.

        Args:
            num_itr (int): The number of SOR-Chebyshev iterations to perform.

        """
        super(SOR_CHEBY, self).__init__(n, A, H, bs, y)
        self.omega = torch.tensor(omega)
        self.omegaa = torch.tensor(omegaa)
        self.gamma = torch.tensor(gamma)

    def iterate(self, num_itr):
        """Perform the SOR-Chebyshev iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        
        inv_omega = torch.div(1, self.omega)
        invM_sor = torch.linalg.inv(self.D - torch.mul(inv_omega, self.L))

        s = torch.zeros(self.bs, self.n).to(device)
        s_new = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        s_present = s
        s_old = torch.zeros(s_present.shape).to(device)

        for i in range(num_itr):
            temp = torch.mul((inv_omega - 1), self.D) + torch.mul((inv_omega), self.U)
            s = torch.matmul(s, torch.matmul(invM_sor, temp)) + torch.matmul(yMF, invM_sor)

            s_new = self.omegaa * (self.gamma * (s - s_present) + (s_present - s_old)) + s_old
            s_old = s
            s_present = s_new

            traj.append(s_new)

        return s_new, traj

In [51]:
class AOR(base_model):
    """Class implementing the Accelerated Over-Relaxation (AOR) algorithm for solving a linear system.

    Args:
        num_itr (int): The number of AOR iterations to perform.

    Attributes:
        num_itr (int): The number of AOR iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the AOR iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y, omega:float=0.3, r:float=0.2):
        """Initialize the AOR solver.

        Args:
            num_itr (int): The number of AOR iterations to perform.

        """
        super(AOR, self).__init__(n, A, H, bs, y)
        self.omega = torch.tensor(omega)
        self.r = torch.tensor(r)

    def iterate(self, num_itr):
        """Perform the AOR iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []

        M = (self.D - torch.mul(self.r, self.L))
        invM_aor = torch.linalg.inv(M)
        N = (torch.mul((1 - self.omega), self.D) + torch.mul((self.omega - self.r), self.L) + torch.mul(self.omega, self.U))

        s = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        for i in range(num_itr):
            s = torch.matmul(s, torch.matmul(invM_aor, N)) + torch.mul(self.omega, torch.matmul(yMF, invM_aor))
            traj.append(s)

        return s, traj

class AOR_CHEBY(base_model):
    """Class implementing the Accelerated Over-Relaxation (AOR) with Chebyshev acceleration algorithm for solving a linear system.

    Args:
        num_itr (int): The number of AOR-Chebyshev iterations to perform.

    Attributes:
        num_itr (int): The number of AOR-Chebyshev iterations to perform.

    Methods:
        forward(num_itr, bs, y): Performs the AOR-Chebyshev iterations and returns the final solution.

    """

    def __init__(self, n, A, H, bs, y, omega:float=0.1, r:float=0.1):
        """Initialize the AOR-Chebyshev solver.

        Args:
            num_itr (int): The number of AOR-Chebyshev iterations to perform.

        """
        super(AOR_CHEBY, self).__init__(n, A, H, bs, y)
        self.omega = torch.tensor(omega)
        self.r = torch.tensor(r)
        

    def iterate(self, num_itr):
        """Perform the AOR-Chebyshev iterations and return the final solution.

        Args:
            num_itr (int): The number of iterations to perform.
            bs (int): The batch size.
            y (torch.Tensor): The input tensor of shape (bs, n).

        Returns:
            torch.Tensor: The final solution tensor of shape (bs, n).
            list: List containing the trajectory of solutions throughout the iterations.

        """
        traj = []
        
        s = torch.zeros(self.bs, self.n).to(device)
        traj.append(s)

        yMF = torch.matmul(self.y, self.H.T)  # Assuming H is defined
        s = torch.matmul(yMF, self.Dinv)  # Generate batch initial solution vector

        Y0 = s

        M = (self.D - torch.mul(self.r, self.L))
        invM = torch.linalg.inv(M)
        N = (torch.mul((1 - self.omega), self.D) + torch.mul((self.omega - self.r), self.L) + torch.mul(self.omega, self.U))
        temp = torch.matmul(invM, N)

        rho = torch.tensor(0.1)
        mu0 = torch.tensor(1)
        mu1 = rho
        xhat1 = torch.matmul(s, temp) + self.omega * torch.matmul(yMF, invM)
        Y1 = xhat1
        Y = Y1

        for i in range(num_itr):
            f = 2 / (rho * mu1)
            j = 1 / mu0
            c = f - j
            mu = 1 / c
            a = (2 * mu) / (rho * mu1)
            Y = torch.matmul((Y1 * a), torch.matmul(invM, N)) - (((mu / mu0)) * Y0) + (a * torch.matmul(yMF, invM))
            Y0 = Y1
            Y1 = Y
            mu0 = mu1
            mu1 = mu

            traj.append(Y)

        return Y, traj

In [39]:
# Model parameters and Parameters for evaluation of generalization error
total_itr = 25  # Total number of iterations
n = 300  # Size of matrix (rows) # ? Paramètre non ?
m = 600  # Size of matrix (columns) # ? Paramètre non ?
bs = 10  # Number of samples #? comment ca samples?

# Generate A and H
A, H, W, solution, y = generate_A_H_sol(n=n, m=m, seed=12, bs=bs)
# Decmposed matrix calculations
#A, D, L, U, Dinv, Minv = decompose_matrix(A)

Condition number, min. and max. eigenvalues of A:
30.828752475024576 5.678370598467175 0.18419073567986302


## Gauss Seidel

In [40]:
gs_model = GS(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_GS = model_iterations(total_itr = total_itr, n = n, bs = bs, model = gs_model, solution = solution)
norm_list_GS

[0.5542690022786458,
 0.29969219970703126,
 0.091330078125,
 0.04125645955403646,
 0.020890432993570962,
 0.011263467152913412,
 0.006311164855957031,
 0.00362995179494222,
 0.0021278618176778156,
 0.0012654950618743897,
 0.0007612222830454509,
 0.00046209959189097086,
 0.00028262742360432943,
 0.00017393902937571208,
 0.0001076107124487559,
 6.687311331431072e-05,
 4.1717588901519774e-05,
 2.611249933640162e-05,
 1.6393260409434637e-05,
 1.0318728784720103e-05,
 6.510551397999128e-06,
 4.116682025293509e-06,
 2.6081756999095282e-06,
 1.6553993336856366e-06,
 1.052458304911852e-06,
 6.701759217927853e-07]

## RI

In [41]:
ri_model = RI(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_RI = model_iterations(total_itr = total_itr, n = n, bs = bs, model = ri_model, solution = solution)
norm_list_RI

[0.5542690022786458,
 0.16198585001627605,
 0.10749448649088542,
 0.08148813883463542,
 0.06404676818847656,
 0.051292872111002606,
 0.041642117818196614,
 0.03418036651611328,
 0.028313741048177084,
 0.023636802673339844,
 0.019864117940266927,
 0.016789969126383465,
 0.014263055165608723,
 0.012170079549153646,
 0.010424921035766602,
 0.0089611873626709,
 0.007727055867513021,
 0.006681643803914388,
 0.005792380015055339,
 0.005033082644144694,
 0.0043825416564941405,
 0.003823445638020833,
 0.003341583251953125,
 0.0029252071380615236,
 0.002564564863840739,
 0.0022515152295430503]

## Jacobi

In [45]:
Jacobi_model = RI(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_Jacobi = model_iterations(total_itr = total_itr, n = n, bs = bs, model = Jacobi_model, solution = solution)
norm_list_Jacobi

[0.5542690022786458,
 0.16198585001627605,
 0.10749448649088542,
 0.08148813883463542,
 0.06404676818847656,
 0.051292872111002606,
 0.041642117818196614,
 0.03418036651611328,
 0.028313741048177084,
 0.023636802673339844,
 0.019864117940266927,
 0.016789969126383465,
 0.014263055165608723,
 0.012170079549153646,
 0.010424921035766602,
 0.0089611873626709,
 0.007727055867513021,
 0.006681643803914388,
 0.005792380015055339,
 0.005033082644144694,
 0.0043825416564941405,
 0.003823445638020833,
 0.003341583251953125,
 0.0029252071380615236,
 0.002564564863840739,
 0.0022515152295430503]

## SOR

In [46]:
SOR_model = SOR(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_SOR = model_iterations(total_itr = total_itr, n = n, bs = bs, model = SOR_model, solution = solution)

norm_list_SOR

[0.5542690022786458,
 1.6296131184895832,
 1.8272568359375,
 1.6980481770833333,
 1.6452791341145834,
 1.6645615234375,
 1.6672140299479166,
 1.66648974609375,
 1.6658727213541666,
 1.6658981119791667,
 1.6659560546875,
 1.6659881184895833,
 1.6659694010416666,
 1.66596728515625,
 1.6659685872395833,
 1.6659694010416666,
 1.6659694010416666,
 1.6659694010416666,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334]

## SOR Chebychev

In [47]:
SOR_Cheby_model = SOR_CHEBY(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_SOR_Cheby = model_iterations(total_itr = total_itr, n = n, bs = bs, model = SOR_Cheby_model, solution = solution)
norm_list_SOR_Cheby

[0.9992316080729167,
 0.9003361002604167,
 1.5777154947916667,
 1.6990768229166666,
 1.6634928385416667,
 1.6603639322916666,
 1.66555517578125,
 1.666478515625,
 1.6660919596354167,
 1.6659236653645833,
 1.6659391276041666,
 1.66597412109375,
 1.6659736328125,
 1.6659685872395833,
 1.6659685872395833,
 1.6659694010416666,
 1.6659690755208334,
 1.6659685872395833,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334,
 1.6659690755208334]

## AOR

In [48]:
AOR_model = AOR(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_AOR = model_iterations(total_itr = total_itr, n = n, bs = bs, model = AOR_model, solution = solution)
norm_list_AOR

[0.5542690022786458,
 1.4047508138020834,
 2.9644684244791666,
 5.620901692708333,
 9.988266927083334,
 17.042154947916668,
 28.3375703125,
 46.361953125,
 75.10869270833334,
 121.01108333333333,
 194.46829166666666,
 312.34372916666666,
 502.0659166666667,
 808.3736666666666,
 1304.4303333333332,
 2110.1786666666667,
 3422.6893333333333,
 5566.440333333333,
 9076.721333333333,
 14838.172,
 24315.237333333334,
 39935.882666666665,
 65731.30666666667,
 108403.52,
 179108.50133333332,
 296437.7386666667]

## AOR Chebychev

In [52]:
AOR_Cheby_model = AOR_CHEBY(n = n, A = A, H = H, bs = bs, y = y)

s_hat, norm_list_AOR_Cheby = model_iterations(total_itr = total_itr, n = n, bs = bs, model = AOR_Cheby_model, solution = solution)
norm_list_AOR_Cheby

[0.7679833170572916,
 4.285617513020833,
 11.545651041666666,
 22.693388020833332,
 37.9719296875,
 57.72351041666667,
 82.39350520833334,
 112.53766666666667,
 148.83254166666666,
 192.08972916666667,
 243.27289583333334,
 303.5192083333333,
 374.1655,
 456.77866666666665,
 553.1919166666667,
 665.5481666666667,
 796.3508333333333,
 948.5225833333334,
 1125.47775,
 1331.202,
 1570.3518333333334,
 1848.3668333333333,
 2171.605,
 2547.499333333333,
 2984.7426666666665,
 3493.509]